### 1. Establishing a connection between python and MySQL

In [ ]:
#Importing necessary libraries 
import mysql.connector
from mysql.connector import Error
import pandas as pd

### 2. Ingesting data from the dataset into multiple tables of mysql database

In [ ]:
try:
    conn = mysql.connector.connect(host='localhost',
                                   database='sports_database',
                                   user='root',
                                   password='123mysql.',
                                   port='3307')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        #creating Player_details table
        cursor.execute('DROP TABLE IF EXISTS Player_details;')
        cursor.execute("CREATE TABLE Player_details (Sl_no Integer PRIMARY KEY, Player VARCHAR(255) NOT NULL,Team VARCHAR(255), Age INTEGER,Height REAL,Weight REAL);")
        print("Player_details table is created")
        
        #creating Player_stats table
        cursor.execute('DROP TABLE IF EXISTS Player_stats;')
        cursor.execute("CREATE TABLE Player_stats (Sl_no Integer, Player VARCHAR(255) NOT NULL,Position VARCHAR(255),Goals INTEGER,Assists INTEGER,YellowCards INTEGER,RedCards INTEGER,PassCompletionRate INTEGER,DistanceCovered INTEGER,Sprints INTEGER,ShotsOnTarget INTEGER,TacklesWon INTEGER,CleanSheets INTEGER,Season INTEGER,PerformanceRatio INTEGER,PredictedPerformanceCategory VARCHAR(255),PressurePerformanceImpact INTEGER,FOREIGN KEY(Sl_no) REFERENCES player_details(Sl_no));")
        print("Player_stats table is created")
       
        #creating Training table
        cursor.execute('DROP TABLE IF EXISTS Training;')
        cursor.execute("CREATE TABLE Training (Sl_no Integer , Player VARCHAR(255) NOT NULL,TrainingHours INTEGER,EffectiveTraining INTEGER,EffectiveTrainingHours INTEGER,Season INTEGER,FOREIGN KEY(Sl_no) REFERENCES Player_Details(Sl_no));")
        print("Training table is created")
       
        #creating Player_PErfromance table
        cursor.execute('DROP TABLE IF EXISTS Player_Performance;')
        cursor.execute("CREATE TABLE Player_performance (Sl_no INTEGER , Player VARCHAR(255), PlayerFatigue INTEGER,MatchPressure INTEGER ,InjuryHistory INTEGER,FatigueInjuryCorrelation INTEGER ,PressurePerformanceImpact INTEGER,Season INTEGER,PerformanceRatio INTEGER,PredictedPerformanceCategory VARCHAR(255),FatiguePressureInteraction INTEGER,PredictedInteractionCategory VARCHAR(255),FOREIGN KEY(Sl_no) REFERENCES Player_Details(Sl_no));")
        print("Player_performance table is created")
        
        # Assuming playersData is your pandas DataFrame containing player data
        playersData = pd.read_csv("cleaned_sports_dataset.csv")
        
        for i, row in playersData.iterrows():
            sql = "INSERT INTO Player_details (Sl_no, Player,Team,Age,Height, Weight) VALUES (%s, %s,%s, %s,%s, %s)"
            cursor.execute(sql, (row['Sl_no'], row['Player'],row['Team'],row['Age'],row['Height'],row['Weight']))  # Adjust column names as needed
            print("Record inserted")
            # The connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
        
        for j, row in playersData.iterrows():
            sql = "INSERT INTO Player_stats (Sl_no, Player,Team,Position,Goals,Assists,YellowCards,RedCards,PassCompletionRate,DistanceCovered,Sprints,ShotsOnTarget,TacklesWon,CleanSheets,Season,PerformanceRatio,PredictedPerformanceCategory,PressurePerformanceImpact) VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s)"
            cursor.execute(sql, (row['Sl_no'], row['Player'],row['Team'],row['Position'],row['Goals'],row['Assists'],row['YellowCards'],row['RedCards'],row['PassCompletionRate'],row['DistanceCovered'],row['Sprints'],row['ShotsOnTarget'],row['TacklesWon'],row['CleanSheets'],row['Season'],row['PerformanceRatio'],row['PredictedPerformanceCategory'],row['PressurePerformanceImpact']))  # Adjust column names as needed
            print("Record inserted")
            # The connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
        
        for k, row in playersData.iterrows():
            sql = "INSERT INTO Training (Sl_no, Player,TrainingHours,EffectiveTraining,EffectiveTrainingHours,Season) VALUES (%s, %s,%s, %s,%s, %s)"
            cursor.execute(sql, (row['Sl_no'], row['Player'],row['TrainingHours'],row['EffectiveTraining'],row['EffectiveTrainingHours'],row['Season']))  # Adjust column names as needed
            print("Record inserted")
            # The connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
        
        for l, row in playersData.iterrows():
            sql = "INSERT INTO Player_performance (Sl_no, Player,PlayerFatigue,MatchPressure,InjuryHistory,FatigueInjuryCorrelation ,PressurePerformanceImpact,Season ,PerformanceRatio,PredictedPerformanceCategory,FatiguePressureInteraction,PredictedInteractionCategory) VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s)"
            cursor.execute(sql, (row['Sl_no'], row['Player'],row['PlayerFatigue'],row['MatchPressure'],row['InjuryHistory'],row['FatigueInjuryCorrelation'],row['PressurePerformanceImpact'],row['Season'],row['PerformanceRatio'],row['PredictedPerformanceCategory'],row['FatiguePressureInteraction'],row['PredictedInteractionCategory']))  # Adjust column names as needed
            print("Record inserted")
            # The connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
            
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed")

### 3. Implementation of CTEs

In [ ]:
#An example of CTE for representinf top team in the dataset
def create_top_team_cte():
    try:
        # Connect to MySQL
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="123mysql.",
            database="sports_database",
            port='3307'
        )

        # Create cursor
        cursor = conn.cursor()

        # Define the CTE query to select top players
        cte_query = """
            WITH TopTeam AS (
            SELECT
                Sl_no,
                Player,
                Team,
                Goals,
                Season,
                ROW_NUMBER() OVER (ORDER BY Goals DESC) AS team_rank
            FROM
                sports_dataset
        )
        SELECT
            Sl_no,
            Player,
            Team,
            Goals,
            team_rank
        FROM
            TopTeam
        WHERE
            team_rank <= 10

        """

        # Execute the CTE query
        cursor.execute(cte_query)

        # Fetch the results
        top_team = cursor.fetchall()

        # Print the top teams
        print("Top Team:")
        for team in top_team:
            print(team)

    except mysql.connector.Error as error:
        print("Error:", error)

    finally:
        # Close the connection
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection is closed")

# Call the function to create the CTE for top team
create_top_team_cte()
print('top_team_cte created')

### 4. Implementing data security and access control mechanisms

In [ ]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123mysql.",
    database="sports_database",
    port='3307'
)

# Create cursor
cursor = conn.cursor()

# Create a new user
def create_user(username, password):
    query = f"CREATE USER '{username}'@'localhost' IDENTIFIED BY '{password}'"
    cursor.execute(query)
    print(f"User {username} created successfully")

# Grant privileges to the user
def grant_privileges(username, database):
    query = f"GRANT ALL PRIVILEGES ON {database}.* TO '{username}'@'localhost'"
    cursor.execute(query)
    print(f"Privileges granted to {username}")

# Grant specific privileges to the user
def grant_specific_privileges(username, database, privileges):
    query = f"GRANT {privileges} ON {database}.* TO '{username}'@'localhost'"
    cursor.execute(query)
    print(f"{privileges} privileges granted to {username}")
    
# Revoke privileges from the user
def revoke_privileges(username, database):
    query = f"REVOKE ALL PRIVILEGES ON {database}.* FROM '{username}'@'localhost'"
    cursor.execute(query)
    print(f"Privileges revoked from {username}")

# Revoke specific privileges from the user
def revoke_specific_privileges(username, database, privileges):
    query = f"REVOKE {privileges} ON {database}.* FROM '{username}'@'localhost'"
    cursor.execute(query)
    print(f"{privileges} privileges revoked from {username}")
    
# Drop user
def drop_user(username):
    query = f"DROP USER '{username}'@'localhost'"
    cursor.execute(query)
    print(f"User {username} dropped successfully")

# Example usage
create_user('new_user1', '1234mysql')
#granting read only privileges
#different privileges include SELECT, INSERT, UPDATE, DELETE, CREATE, DROP, ALTER, GRANT OPTION, ALL PRIVILEGES:
grant_specific_privileges('new_user1', 'sports_database','SELECT')

# Commit the changes
conn.commit()

# Close the connection
conn.close()
